## Dataset used in this notebook
- ratio1.csv, financial.csv, financial 2.csv
- sum_total_releases.csv
- at_df_yret.csv

In [142]:
import pandas as pd

In [143]:
ratio1 = pd.read_csv('ratio1.csv')
financial = pd.read_csv('financial.csv')
financial2 = pd.read_csv('financial 2.csv')

In [144]:
ratio1.head()

,gvkey,adate,qdate,public_date,bm,roa,roe,debt_assets
0,1004,1990-05-31,1990-11-30,1991-01-31,1.201,0.135,0.086,0.507
1,1004,1990-05-31,1990-11-30,1991-02-28,1.201,0.135,0.086,0.507
2,1004,1990-05-31,1990-11-30,1991-03-31,1.201,0.135,0.086,0.507
3,1004,1990-05-31,1991-02-28,1991-04-30,1.113,0.127,0.076,0.504
4,1004,1990-05-31,1991-02-28,1991-05-31,1.113,0.127,0.076,0.504


In [145]:
ratio1_a = ratio1.groupby(['adate', 'gvkey'])[['bm', 'roa', 'roe', 'debt_assets']].mean().reset_index()

In [146]:
ratio1_a['fyear'] = pd.to_datetime(ratio1_a['adate']).dt.year
ratio1_a.head()

,adate,gvkey,bm,roa,roe,debt_assets,fyear
0,1989-12-31,1034,0.596,0.114,0.125,0.671,1989
1,1989-12-31,1075,2.967,0.070,0.032,0.744,1989
2,1989-12-31,1078,0.160,0.333,0.333,0.483,1989
3,1989-12-31,1104,1.107,0.095,-0.018,0.428,1989
4,1989-12-31,1279,1.183,0.139,0.097,0.583,1989


In [147]:
# Merge the result with financial2 on 'gvkey' and 'year'
merged_df = pd.merge(financial, financial2, on=['gvkey', 'fyear'], how='inner')

controls = pd.merge(ratio1_a, merged_df, on=['gvkey', 'fyear'], how='inner')

In [148]:
controls.drop(columns = ['adate', 'datadate', 'indfmt', 'datafmt', 'consol', 'popsrc', 'curcd', 'costat'], 
              axis = 1, inplace=True)


In [149]:
controls_df = controls[controls['exchg'].isin([11, 14, 10])]

In [150]:
controls_df.loc[:, 'IK'] = controls_df['capx'] / controls_df['ppent']
controls_df.loc[:, 'TAN'] = controls_df['ppent'] / controls_df['at']
controls_df.loc[:, 'OL'] = (controls_df['cogs'] + controls_df['xsga']) / controls_df['at']

/var/folders/7m/thggfg6d5178mk1vw5fgt7940000gn/T/ipykernel_44704/1669402566.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  controls_df.loc[:, 'IK'] = controls_df['capx'] / controls_df['ppent']
/var/folders/7m/thggfg6d5178mk1vw5fgt7940000gn/T/ipykernel_44704/1669402566.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  controls_df.loc[:, 'TAN'] = controls_df['ppent'] / controls_df['at']
/var/folders/7m/thggfg6d5178mk1vw5fgt7940000gn/T/ipykernel_44704/1669402566.py:3: SettingWithCopyWarning: 
A value is

In [151]:
sum_p = pd.read_csv('sum_total_releases.csv')

In [152]:
sum_df2_c =controls_df.merge(sum_p, how='inner', left_on=['fyear', 'gvkey'], right_on=['YEAR', 'gvkey'])

In [153]:
sum_df2_c.head()

,gvkey,bm,roa,roe,debt_assets,fyear,at,eq,ppent,sale,exchg,sic,capx,cogs,xsga,IK,TAN,OL,YEAR,SUM_TOTAL_RELEASES
0,4819,1.04000,0.16950,0.06700,0.53775,1991,89.423,71.017,53.048,306.170,11,3050,10.984,208.771,73.491,0.207058,0.593225,3.156481,1991,25105.0
1,5109,1.24900,0.05475,0.00450,0.42025,1991,180.149,140.834,44.965,471.766,11,5661,6.984,292.016,165.756,0.155321,0.249599,2.541074,1991,160474.0
2,5275,1.27225,0.07600,0.03500,0.76975,1991,99.199,54.910,102.998,310.736,11,2750,13.244,225.172,60.916,0.128585,1.038297,2.883981,1991,30000.0
3,7281,0.51550,0.14000,0.11825,0.24300,1991,20.316,26.751,11.232,39.648,11,3564,3.474,27.046,8.231,0.309295,0.552865,1.736415,1991,19792.0
4,8958,0.57300,0.22325,0.18225,0.39575,1991,34.089,30.675,9.947,100.609,14,3080,3.872,78.963,10.971,0.389263,0.291795,2.638212,1991,42380.0


In [154]:
#sum_df2_c.to_csv('sum_df2_c.csv', index=False)

In [155]:
sum_df2_c['p_at'] = sum_df2_c['SUM_TOTAL_RELEASES'] / sum_df2_c['at']

In [156]:
percentiles_df = sum_df2_c.groupby('fyear')['p_at'].quantile([0.2, 0.4, 0.6, 0.8]).unstack().reset_index()
percentiles_df.columns = ['fyear', 'p20', 'p40', 'p60', 'p80']

# Merge percentiles back into the original dataset
sum_df2_c = sum_df2_c.merge(percentiles_df, on='fyear', how='left')

In [157]:
sum_df2_c.head()

,gvkey,bm,roa,roe,debt_assets,fyear,at,eq,ppent,sale,...,IK,TAN,OL,YEAR,SUM_TOTAL_RELEASES,p_at,p20,p40,p60,p80
0,4819,1.04000,0.16950,0.06700,0.53775,1991,89.423,71.017,53.048,306.170,...,0.207058,0.593225,3.156481,1991,25105.0,280.744328,31.899858,203.533654,693.243906,2586.237009
1,5109,1.24900,0.05475,0.00450,0.42025,1991,180.149,140.834,44.965,471.766,...,0.155321,0.249599,2.541074,1991,160474.0,890.784850,31.899858,203.533654,693.243906,2586.237009
2,5275,1.27225,0.07600,0.03500,0.76975,1991,99.199,54.910,102.998,310.736,...,0.128585,1.038297,2.883981,1991,30000.0,302.422403,31.899858,203.533654,693.243906,2586.237009
3,7281,0.51550,0.14000,0.11825,0.24300,1991,20.316,26.751,11.232,39.648,...,0.309295,0.552865,1.736415,1991,19792.0,974.207521,31.899858,203.533654,693.243906,2586.237009
4,8958,0.57300,0.22325,0.18225,0.39575,1991,34.089,30.675,9.947,100.609,...,0.389263,0.291795,2.638212,1991,42380.0,1243.216287,31.899858,203.533654,693.243906,2586.237009


In [158]:
# Function to map e_ value to group tag
def map_to_group_tag(scale_value, percentiles):
    if scale_value <= percentiles['p20']:
        return 1
    elif scale_value <= percentiles['p40']:
        return 2
    elif scale_value <= percentiles['p60']:
        return 3
    elif scale_value <= percentiles['p80']:
        return 4
    else:
        return 5  # Adjust this based on your requirements

# Add a new column 'group_tag' to the DataFrame based on 'e_at' percentiles for each year
group_tags = []

for year in sum_df2_c['fyear'].unique():
    year_data = sum_df2_c[sum_df2_c['fyear'] == year]
    percentiles = year_data[['p20', 'p40', 'p60', 'p80']].iloc[0].to_dict()
    group_tags.extend(year_data['p_at'].apply(map_to_group_tag, args=(percentiles,)))

sum_df2_c['group_tag'] = group_tags

In [159]:
#sum_df2_c.to_csv('control+p_at.csv', index=False)


In [160]:
at_df_yret = pd.read_csv('at_df_yret.csv')

In [161]:
at_df_yret = pd.merge(at_df_yret, sum_df2_c, how='left', left_on=['fyear','gvkey'],right_on=['fyear','gvkey'])

In [162]:
at_df_yret.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10838 entries, 0 to 10837
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fyear               10838 non-null  int64  
 1   gvkey               10838 non-null  int64  
 2   at_x                10838 non-null  float64
 3   p_at_x              10838 non-null  float64
 4   group_tag_x         10838 non-null  object 
 5   trade               10838 non-null  int64  
 6   weight_l            10838 non-null  float64
 7   weight_s            10838 non-null  float64
 8   weight              10838 non-null  float64
 9   yret                10838 non-null  float64
 10  cap                 10838 non-null  float64
 11  rf                  10838 non-null  float64
 12  Implied ERP         10838 non-null  float64
 13  ret_prem            10838 non-null  float64
 14  bm                  10294 non-null  float64
 15  roa                 10438 non-null  float64
 16  roe 

In [168]:

import numpy as np
at_df_yret['eq2'] = np.log(at_df_yret['eq'])


/Users/liuqiaochu/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/liuqiaochu/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [166]:
grouped_avg = at_df_yret.groupby('group_tag_y')[['SUM_TOTAL_RELEASES','p_at_y','bm','roa','roe','eq','eq2','OL','IK','TAN','debt_assets']].mean()
grouped_avg

,SUM_TOTAL_RELEASES,p_at_y,bm,roa,roe,eq,eq2,OL,IK,TAN,debt_assets
group_tag_y,,,,,,,,,,,
1.0,1.596863e+03,0.746586,0.532582,0.153469,0.143604,3211.977686,6.541253,2.177871,0.195985,0.694791,0.524703
2.0,2.927895e+04,14.864700,0.567008,0.149402,0.109171,4374.869847,6.864789,1.972553,0.190766,0.754504,0.528079
3.0,1.882629e+05,97.595285,0.625541,0.149693,0.132637,3381.006684,6.739695,2.225393,0.180838,0.898627,0.542101
4.0,8.986821e+05,487.112910,0.598353,0.152871,0.127379,4075.800955,NaN,2.666909,0.164689,1.186832,0.545112
5.0,8.445205e+06,16001.383871,0.704271,0.134508,0.090028,2413.297244,6.595575,2.809034,0.141260,2.607503,0.605001
